#  Two Dimensional Galactic Orbits

* set initial conditions (x0,y0) and (vx0,vy0) in the plane z=0
* set integration time step
* set number of integrations or a final integratop stop time
* define the potential and derive the forces

In [ ]:
import numpy as np
import math
%matplotlib inline
import matplotlib.pyplot as plt

The Plummer potential for mass $M$ and core radius $r_c$ is given by
$$
\Phi = -  {  M  \over {   {(r_c^2 + r^2)}^{1/2} }  }
$$
and is also used to described softened gravity of a point mass (think of the case $r_c = 0$).


The force is the gradient of the potential
$$
f = -\nabla \Phi
$$

We also want to record the total energy (kinetic and potential):
$$
E = { 1\over 2} v^2 + \Phi
$$
and angular momentum
$$
J = r \times v
$$

### force field 

In [ ]:
def radius(x):
    return math.sqrt(np.inner(x,x))

def potential(pos):
    r = radius(pos)
    y1 = 1+r*r
    return -1.0/math.sqrt(y1)

def angmomz(pos,vel):
    return pos[0]*vel[1] - pos[1]*vel[0]

def energy(pos,vel):
    return 0.5*np.inner(vel,vel) + potential(pos)

def force(pos):
    r = radius(pos)
    y2 = 1.0/math.sqrt(1+r*r)
    return -pos*y2*y2*y2

### Euler integrator

In [ ]:
def step(pos,vel, dt):
    pos = pos + dt*vel
    vel = vel + dt*force(pos)
    return (pos,vel)

def step(pos,vel, dt):
    vel = vel + dt*force(pos)
    pos = pos + dt*vel
    return (pos,vel)


### Helpher functions

In [ ]:
def show_stats(data):
    m = data.mean()
    s = data.std()
    dmin = data.min()
    dmax = data.max()
    rmin = (dmin-m)/s
    rmax = (dmax-m)/s
    print("Mean/Std:",m,s,s/m)
    print("Min/Max:",dmin,dmax)
    print("Rmin/Rmax:",rmin,rmax)

### Initial conditions

For 2D orbits we only specify the X coordinate and Y velocity. The remaining values of the 6 phase space coordinates are 0. Why is this?


In [ ]:
x0 = 1.0
v0 = 0.1
n = 200
dt = 0.1

#
t = 0.0
pos = np.array([x0,  0.0, 0.0])
vel = np.array([0.0,  v0, 0.0])
time = np.zeros(1)
e = energy(pos,vel)
j = angmomz(pos,vel)
phase = np.concatenate(([t,e,j],pos,vel)).reshape(1,9)

### Integrate


In [ ]:
%%time
for i in range(n):
    (pos,vel) = step(pos,vel,dt)
    t = t + dt
    e = energy(pos,vel)
    j = angmomz(pos,vel)
    #print(i,pos,vel)
    p = np.concatenate(([t,e,j],pos,vel)).reshape(1,9)
    phase = np.concatenate((phase, p),axis=0)
    time = np.append(time,t)
#print(phase)

In [ ]:
plt.scatter(phase[:,3],phase[:,4],c=time)
plt.title("Orbit")

In [ ]:
plt.scatter(phase[:,0], phase[:,1])
plt.title("Conserving Energy?")
show_stats(phase[:,1])

In [ ]:
plt.scatter(phase[:,0], phase[:,2])
plt.title("Conserving Angular Momentum?")
show_stats(phase[:,2])

### Saving data

There are many good and less ideal ways to save data. In astronomy standard formats such has FITS and HDF5 are common.  For our work here we use a simple and fast native python method, called pickle. You can save whole objects, and reading them back in will ensure the whole object structure and hierarchy is preserved. 

In [ ]:
try:
    import cPickle as pickle
    print("using cPickle")
except:
    import pickle
    print("using pickle")

pdata = pickle.dumps(phase)
#
phase2 = pickle.loads(pdata)

print(phase[0])
print(phase2[0])

# now write to, and read from, a file on disk

# Questions

* If we are just doing two dimensional orbits, can't we just leave the Z off and speed up computations? What do you need to change to do this?
* If want to squash the potential and make it slightly oval, what would the changes be. Here we would define an ellipsoidal radius on which the potential is constant:
$$
r^2 = { x^2 \over a^2} + { y^2 \over b^2 }
$$
instead of the normal 
$$
r^2 = x^2 + y^2
$$
* A 2009 IAS lecture by Tremaine is an excellent lecture for (symplectic) orbit integrators

### Orbits using scipy

For many scientific applications there are canned routines made available by the community.  The **scipy** package is one such module. We will derive the same orbit integration using **scipy.odeint**

In [ ]:
from scipy.integrate import odeint



In [ ]:
def ofunc(y,t):
    pos = y[0:3]
    vel = y[3:]
    return np.concatenate((vel,force(pos)))
  


In [ ]:
phase0 = np.array([x0,0,0, 0,v0,0])
times  = np.arange(0.0,(n+1)*dt,dt)
print(ofunc(phase0,0.0))

In [ ]:
%%time
orbit = odeint(ofunc, phase0, times)


In [ ]:
plt.scatter(orbit[:,0],orbit[:,1],c=time)
#plt.scatter(phase[:,3],phase[:,4])
plt.title("Orbit")

In [ ]:
p1 = phase[-1,3:]
p2 = orbit[-1,:]
#
print(p1)
print(p2)
print(p1-p2)